In [1]:
import pandas as pd
import sys
sys.path.append('C:\\python_source')
import NERP_PI_LC

sessions = NERP_PI_LC.check_and_open_sap('SEP', 'ID', 'PW',3)
session = sessions[1]

def crawl_hscode_with_sr(session, sr_no):
    dict_result = dict()
    session.StartTransaction('ZLED50110_D')
    session.findById("wnd[0]/usr/ctxtG_TKNUM").text = sr_no #"9102399235"
    session.findById("wnd[0]").sendVKey(0)

    # Sales, Plant
    dict_result['SalesOrg'] = session.findById("wnd[0]/usr/txtGT_SR-ZZVKORG").text
    dict_result['PlantCode'] = session.findById("wnd[0]/usr/txtGT_SR-ZZWERKS").text
    dict_result['PlantName'] = session.findById("wnd[0]/usr/txtGT_SR-WERKS_NAME").text

    # Port
    session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC1").select()
    dict_result['POL'] = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC1/ssubSUB_SCA:SAPMZLED50110:0310/txtG_LOAD_TEXT").text
    dict_result['F.Dest'] = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC1/ssubSUB_SCA:SAPMZLED50110:0310/txtG_FDEST_TEXT").text

    session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4").select()

    # [Material&Container탭 상단] HSCODE, DESCRIPTION 
    temp_set_hscode = set()
    temp_set_itemname = set()
    temp_set_materialcode = set()
    temp_set_totalpkg = set()
    total_row = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A").RowCount
    for idx in range((total_row)):
        session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A").verticalScrollbar.position = idx
        temp_set_hscode.add(session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-ZZSTAWN[14,{0}]").text)
        temp_set_itemname.add(session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-GOODS_DESC1[13,{0}]").text)
        temp_set_materialcode.add(session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-PRODUCT_ID[2,{0}]").text)
        txt_plt = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/txtGT_SR-ZZPACK_QTY2").text
        txt_plt_type = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/ctxtGT_SR-ZZPACK_QTY2_UOM").text
        txt_car = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/txtGT_SR-ZZPACK_QTY").text
        txt_car_type = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/ctxtGT_SR-ZZPACK_QTY_UOM").text
        #print(type(txt_plt))
        temp_set_totalpkg.add(f"{'' if txt_plt == '0' else txt_plt}{txt_plt_type} {txt_car}{txt_car_type}".strip())

    # Special Code for X010
    if dict_result['SalesOrg'] == 'X010':
        temp_set_x010_material = set()
        for idx in range((total_row)):
            session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A").verticalScrollbar.position = idx
            txt_itemname = session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-GOODS_DESC1[13,{0}]").text
            txt_materialcode = session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340A/txtGT_MAT-PRODUCT_ID[2,{0}]").text
            temp_set_x010_material.add(f"{txt_itemname}\n{txt_materialcode}")
         
    dict_result['HSCODE'] = ', '.join(temp_set_hscode)
    dict_result['MaterialCode'] = ', '.join(temp_set_materialcode)
    dict_result['Description'] = ', '.join(temp_set_itemname)
    dict_result['Total PKG'] = ', '.join(temp_set_totalpkg)

    # [Material&Container탭 하단] Container
    temp_set_container = set()
    total_row = session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340B").RowCount
    for idx in range((total_row)):
        session.findById("wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340B").verticalScrollbar.position = idx
        temp_set_container.add(session.findById(f"wnd[0]/usr/tabsSUB/tabpSUB_FC4/ssubSUB_SCA:SAPMZLED50110:0340/tblSAPMZLED50110TC_0340B/txtGT_CONT-CNT_NO[0,{0}]").text)

    dict_result['ContainerNo'] = ', '.join(temp_set_container)

    # Special Code for X010
    if dict_result['SalesOrg'] == 'X010':
        dict_result['X010_Material'] = '\n'.join(temp_set_x010_material)   


    session.findById("wnd[0]").sendVKey(3)

    return dict_result

## 1건 확인용

In [2]:
# SR 1건 확인
sr_no = '9102755318'
data_all = crawl_hscode_with_sr(session, sr_no)

# 엑셀저장용으로 데이터프레임 생성
df = pd.DataFrame(columns=[list(data_all.keys())])
for each_col in data_all.keys():
    df.loc[sr_no, each_col] = data_all[each_col]

# 결과출력
print('SR NO : ', sr_no)
for key in data_all.keys():
    print(key+' :', data_all[key])

SR NO :  8888855555
SalesOrg : R010
PlantCode : P999
PlantName : Plant
POL : Gwangyang,South Korea
F.Dest : Misurata,Libya
HSCODE : 9999990000, 8888882010
MaterialCode : MW999999, MW999999
Description : MICROWAVE OVEN
Total PKG : 87CAR
ContainerNo : MRKU9999999


## 여러건 확인용
* sr_no의 '''와 ''' 사이에 붙여넣고 실행

In [27]:
# SR 여러건 확인
sr_no = '''
9102622898
9102718369
9102713599
'''.split('\n')

sr_no = list(set(sr_no))
sr_no.remove('')

#빈 DataFrame 생성
df = pd.DataFrame(columns=[list(crawl_hscode_with_sr(session, sr_no[0]).keys())])

for idx, each_sr in enumerate(sr_no):
    data_all = crawl_hscode_with_sr(session, each_sr)
    for each_col in data_all.keys():
        df.loc[each_sr,each_col] = data_all[each_col]

df


,SalesOrg,PlantCode,PlantName,POL,F.Dest,HSCODE,MaterialCode,Description,Total PKG,ContainerNo,X010_Material
9999999898,R010,P999,PLANT,"Gwangyang,South Korea","Tema,Ghana",9999991030,RF9999999,REFRIGERATOR,22PAL,MRKU9999999,NaN
9999999599,X010,B999,PLANT2,"Busan,South Korea","Port Said East,Egypt",9999999000,"BN9900000A, BN9900000A",TV...,1CAR,MRKU9999999,"TV,\nBN99..."
9999999369,R010,P999,PLANT3,"Gwangyang,South Korea","Dakar,Senegal","99999992010, 9999990000, 9999992020","AC9999999, ...",AIRCONDITIONER,333CAR,TGBU9999999,NaN


## 확인한 내용 엑셀로 저장

In [23]:
# 엑셀파일로 저장
from datetime import datetime

df.to_excel(f'hscode_summary_{datetime.strftime(datetime.today(),"%Y%m%d")}.xlsx')